In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px
import seaborn as sns
from matplotlib.pyplot import figure

In [ ]:
dt_bs_train = pd.read_csv('../input/bike-sharing-demand/train.csv')
dt_bs_test = pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
df_uniq_cols = pd.DataFrame(columns = ['columns','values'])
for index, col in enumerate(dt_bs_train.columns.values):
    df_uniq_cols.loc[index, 'columns'] = col
    df_uniq_cols.loc[index, 'values'] = dt_bs_train[col].nunique()
df_uniq_cols

In [ ]:
for col in ['season','holiday','workingday','weather']:
    dt_bs_train[col] = dt_bs_train[col].astype('category')
    dt_bs_test[col] = dt_bs_test[col].astype('category')

In [ ]:
dt_bs_train['datetime'] = pd.to_datetime(dt_bs_train['datetime'])

In [ ]:
figure(figsize=(12, 8), dpi=80)
sns.heatmap(dt_bs_train.corr(), annot = True)

In [ ]:
sns.pairplot(dt_bs_train)

In [ ]:
dt_bs_train['temp'].dtype == 'float64'

In [ ]:
dt_bs_train['year'] = dt_bs_train['datetime'].dt.year
dt_bs_train['month'] = dt_bs_train['datetime'].dt.month
dt_bs_train['day'] = dt_bs_train['datetime'].dt.day
dt_bs_train['hour'] = dt_bs_train['datetime'].dt.hour
dt_bs_train

In [ ]:
px.box(dt_bs_train, y = [cols for cols in dt_bs_train.columns if dt_bs_train[cols].dtype == 'float64'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV


In [ ]:
X = dt_bs_train.drop(columns = ['count','datetime','casual', 'registered'])
y = dt_bs_train['count']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = .2, random_state = 123)

In [ ]:
df_model_rmse = pd.DataFrame(columns = ['model','rmse'])

In [ ]:
pipe = Pipeline(steps = [('scaling', StandardScaler()), ('LR',LogisticRegression())])

pipe.fit(X_train, y_train)

mse = mean_squared_error(pipe.predict(X_test), y_test)

df_model_rmse.loc[0,'model'] = 'Scaled Logistic Regression'
df_model_rmse.loc[0,'rmse'] = mse**(1/2)

print(mse**(1/2))

#### GridSearch

In [ ]:
lr_hyper_tune = {
    'LR__penalty': ['l1','l2'],
    'LR__solver': ['saga','newton-cg', 'lbfgs', 'liblinear', 'sag'] 
}

pipe.get_params()

lr_grid = GridSearchCV(pipe, param_grid = lr_hyper_tune, cv = 5, scoring = 'neg_mean_squared_error')

lr_grid.fit(X_train, y_train)

# lr_grid.best_params_ - {'LR__penalty': 'l1', 'LR__solver': 'saga'}


In [ ]:

lr_grid_best = lr_grid.best_estimator_
mse_lr_best_fit = mean_squared_error(lr_grid_best.predict(X_test), y_test) ** (1/2)
df_model_rmse.loc[1,'model'] = 'GSV Scaled Logistic Regression'
df_model_rmse.loc[1,'rmse'] = mse_lr_best_fit

### AdaBoost

In [ ]:
dt_stump = DecisionTreeRegressor(max_depth = 1)
pipe_adb = Pipeline(steps = [('scaling', StandardScaler()), ('Adb',AdaBoostRegressor(base_estimator = dt_stump, random_state = 123))])

adb_hyper_tune = {
    'Adb__n_estimators': [10, 30, 50, 70, 80, 100],
    'Adb__learning_rate': [0.1, 0.5, 1],
    'Adb__loss': ['linear','square','exponential']
}

param_adb = GridSearchCV(pipe_adb, param_grid = adb_hyper_tune, cv = 5, scoring = 'neg_mean_squared_error')
param_adb.fit(X_train, y_train)

ada_gs_grid_best = param_adb.best_estimator_
mse_ada_gs_best_fit = mean_squared_error(ada_gs_grid_best.predict(X_test), y_test) ** (1/2)
df_model_rmse.loc[2,'model'] = 'GSV Scaled Ada Boost'
df_model_rmse.loc[2,'rmse'] = mse_ada_gs_best_fit

### Bagging Regressor

In [ ]:
dt_stump = DecisionTreeRegressor(max_depth = 1)
pipe_bgr = Pipeline(steps = [('scaling', StandardScaler()), ('Bgr',BaggingRegressor(base_estimator = dt_stump, random_state = 123))])

bgr_hyper_tune = {
    'Bgr__n_estimators': [10, 30, 50, 70, 80, 100, 200, 500],
    'Bgr__max_samples': [0.8, 0.9, 1],
    'Bgr__oob_score': [True]
}

param_bgr = GridSearchCV(pipe_bgr, param_grid = bgr_hyper_tune, cv = 5, scoring = 'neg_mean_squared_error')
param_bgr.fit(X_train, y_train)

bgr_gs_grid_best = param_bgr.best_estimator_
mse_bgr_gs_best_fit = mean_squared_error(bgr_gs_grid_best.predict(X_test), y_test) ** (1/2)
df_model_rmse.loc[3,'model'] = 'GSV Scaled Bgr Boost'
df_model_rmse.loc[3,'rmse'] = mse_bgr_gs_best_fit

### Gradient Boost Regressor

In [ ]:
pipe_grad_boost = Pipeline(steps = [('scaling', StandardScaler()), ('grad_boost',GradientBoostingRegressor(random_state = 123))])

grad_hyper_tune = {
    'grad_boost__n_estimators': [10, 30, 50, 70, 80],
    'grad_boost__max_depth':[3,5,10,15],
    'grad_boost__max_features':['sqrt','log2'],
    'grad_boost__loss': ['ls','lad']
}

param_grad_boost = GridSearchCV(pipe_grad_boost, param_grid = grad_hyper_tune, cv = 5, scoring = 'neg_mean_squared_error')
param_grad_boost.fit(X_train, y_train)

grad_boost_grid_best = param_grad_boost.best_estimator_
grad_boost_gs_best_fit = mean_squared_error(grad_boost_grid_best.predict(X_test), y_test) ** (1/2)
df_model_rmse.loc[4,'model'] = 'GSV Scaled Grad Boost'
df_model_rmse.loc[4,'rmse'] = grad_boost_gs_best_fit

In [ ]:
df_model_rmse

In [ ]:
dt_bs_test['datetime'] = pd.to_datetime(dt_bs_test['datetime'])
dt_bs_test['year'] = dt_bs_test['datetime'].dt.year
dt_bs_test['month'] = dt_bs_test['datetime'].dt.month
dt_bs_test['day'] = dt_bs_test['datetime'].dt.day
dt_bs_test['hour'] = dt_bs_test['datetime'].dt.hour
datetimecol = dt_bs_test['datetime']
dt_bs_test = dt_bs_test.drop(columns = ['datetime'])

In [ ]:
pred_final = np.absolute(grad_boost_grid_best.predict(dt_bs_test))

In [ ]:
submission = pd.DataFrame({
        "datetime": datetimecol,
        "count": pred_final
    })
submission.to_csv('bike_prediction_output.csv', index=False)